In [13]:
from lec_utils import *
def show_merging_animation():
    src = "https://docs.google.com/presentation/d/1HPJ7fiBLNEURsWYiY0qpqPR3qup68Mr0_B34GU99Y8Q/embed?start=false&loop=false&delayms=60000&rm=minimal"
    width = 865
    height = 509
    display(IFrame(src, width, height))

<div class="alert alert-info" markdown="1">

#### Lecture 7

# Aggregation: Reshaping and Merging
    
</div>

### Example: Palmer Penguins

<center><img src="imgs/lter_penguins.png" width=60%>
<i><a href="https://github.com/allisonhorst/palmerpenguins/blob/main/README.md">Artwork by @allison_horst</a></i>

</center>

The dataset we'll work with for the rest of the lecture involves various measurements taken of three species of penguins in Antarctica.

In [14]:
IFrame('https://www.youtube-nocookie.com/embed/CCrNAHXUstU?si=-DntSyUNp5Kwitjm&amp;start=11',
       width=560, height=315)

### Loading the data

In [15]:
penguins = pd.read_csv('../../public_data/penguins.csv')
penguins

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Dream,41.3,20.3,194.0,3550.0,Male
1,Adelie,Torgersen,38.5,17.9,190.0,3325.0,Female
2,Adelie,Dream,34.0,17.1,185.0,3400.0,Female
...,...,...,...,...,...,...,...
330,Chinstrap,Dream,46.6,17.8,193.0,3800.0,Female
331,Adelie,Dream,39.7,17.9,193.0,4250.0,Male
332,Gentoo,Biscoe,45.1,14.5,207.0,5050.0,Female


- Here, each row corresponds to a single penguin, and each column corresponds to a different attribute (or feature) we have for each penguin.

- Data formatted in this way is sometimes called [tidy data](https://r4ds.had.co.nz/tidy-data.html).

### Visualizing the data

In [16]:
penguins.plot(
    kind="scatter",
    x="bill_length_mm",
    y="body_mass_g",
    color="species",
    title="Body Mass vs. Bill Length",
)

#### Warmup: What is the mean `'body_mass_g'` for all penguins?

In [17]:
penguins.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Dream,41.3,20.3,194.0,3550.0,Male
1,Adelie,Torgersen,38.5,17.9,190.0,3325.0,Female
2,Adelie,Dream,34.0,17.1,185.0,3400.0,Female
3,Chinstrap,Dream,45.5,17.0,196.0,3500.0,Female
4,Adelie,Biscoe,39.7,18.9,184.0,3550.0,Male


In [18]:
penguins['body_mass_g'].mean() 

4207.057057057057

#### Warmup: What is the mean `'body_mass_g'` **for each `'species'`**?

In [19]:
penguins.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Dream,41.3,20.3,194.0,3550.0,Male
1,Adelie,Torgersen,38.5,17.9,190.0,3325.0,Female
2,Adelie,Dream,34.0,17.1,185.0,3400.0,Female
3,Chinstrap,Dream,45.5,17.0,196.0,3500.0,Female
4,Adelie,Biscoe,39.7,18.9,184.0,3550.0,Male


In [20]:
# To find the mean 'body_mass_g' for each 'species':
penguins.groupby('species')['body_mass_g'].mean() 

species
Adelie       3706.16
Chinstrap    3733.09
Gentoo       5092.44
Name: body_mass_g, dtype: float64

### Reminder: Column independence

- As we've seen, within each group, the aggregation method is applied to **each column independently**.

In [21]:
penguins.groupby('species').max()

,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
species,,,,,,
Adelie,Torgersen,46.0,21.5,210.0,4775.0,Male
Chinstrap,Dream,58.0,20.8,212.0,4800.0,Male
Gentoo,Biscoe,59.6,17.3,231.0,6300.0,Male


- The above result **is not** telling us that there is a `'Adelie'` penguin with a `'body_mass_g'` of `4775.0` that lived on `'Torgersen'` island.

In [22]:
# This penguin lived on Biscoe island!
penguins.loc[(penguins['species'] == 'Adelie') & (penguins['body_mass_g'] == 4775.0)]

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
255,Adelie,Biscoe,43.2,19.0,197.0,4775.0,Male


### `agg` with a custom aggregation method

- **Example**: What is the **second largest** recorded `'body_mass_g'` for each `'species'`?

In [23]:
# Here, the argument to agg is a function,
# which takes in a Series and returns a scalar.
def second_largest(s):
    return s.sort_values().iloc[-2]
(
    penguins
    .groupby('species')
    ['body_mass_g']
    .agg(second_largest)
)

species
Adelie       4725.0
Chinstrap    4550.0
Gentoo       6050.0
Name: body_mass_g, dtype: float64

- Note: If you give `agg` a custom function, it should map $\texttt{Series} \rightarrow \texttt{number}$.    

In [24]:
# How to write this without *defining* a function, if we won't use it again?




### Grouping with multiple columns

- When we group with multiple columns, one group is created for **every unique combination** of elements in the specified columns.<br><small>In the output below, why are there only 5 rows, rather than $3 \times 3 = 9$ rows, when there are 3 unique `'species'` and 3 unique `'island'`s?</small>

In [25]:
# Read this as:
species_and_island = (
    penguins.groupby(['species', 'island'])         # for every combination of 'species' and 'island' in the DataFrame,
    [['bill_length_mm', 'bill_depth_mm']].mean()    # calculate the mean 'bill_length_mm' and the mean 'bill_depth_mm'.
)
species_and_island

bill_length_mm  bill_depth_mm
species   island                                  
Adelie    Biscoe              38.98          18.37
          Dream               38.52          18.24
          Torgersen           39.04          18.45
Chinstrap Dream               48.83          18.42
Gentoo    Biscoe              47.57          15.00

- **Advice**: When grouping on multiple columns, the result usually has a `MultiIndex`;  use `reset_index` or set `as_index=False` in `groupby` to avoid this.

In [26]:
# Now, this looks like a regular DataFrame!
species_and_island.reset_index() 

,species,island,bill_length_mm,bill_depth_mm
0,Adelie,Biscoe,38.98,18.37
1,Adelie,Dream,38.52,18.24
2,Adelie,Torgersen,39.04,18.45
3,Chinstrap,Dream,48.83,18.42
4,Gentoo,Biscoe,47.57,15.00


In [27]:
# What about the mean 'body_mass_g' for each 'species' and 'sex'?
(
    penguins.groupby(['species', 'sex'])
    [['body_mass_g']].mean()
)

body_mass_g
species   sex                
Adelie    Female      3368.84
          Male        4043.49
Chinstrap Female      3527.21
          Male        3938.97
Gentoo    Female      4679.74
          Male        5484.84

## Pivot tables using `pivot_table`

---

### Pivot tables: An extension of grouping

- Pivot tables are a compact way to display tables for humans to read.

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>sex</th>
      <th>Female</th>
      <th>Male</th>
    </tr>
    <tr>
      <th>species</th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>Adelie</th>
      <td>3368.84</td>
      <td>4043.49</td>
    </tr>
    <tr>
      <th>Chinstrap</th>
      <td>3527.21</td>
      <td>3938.97</td>
    </tr>
    <tr>
      <th>Gentoo</th>
      <td>4679.74</td>
      <td>5484.84</td>
    </tr>
  </tbody>
</table>


- Notice that each value in the table is the average of `'body_mass_g'` of penguins, for every combination of `'species'` and `'sex'`.

- **You can think of pivot tables as grouping using two columns, then "pivoting" one of the group labels into columns.**

### `pivot_table`

- The `pivot_table` DataFrame method aggregates a DataFrame using two columns. To use it:
<br><br>
```python
        df.pivot_table(index=index_col,
                       columns=columns_col,
                       values=values_col,
                       aggfunc=func)
```

- The resulting DataFrame will have:
    - One row for every unique value in `index_col`.
    - One column for every unique value in `columns_col`.
    - Values determined by applying `func` on values in `values_col`.

- **Example**: Find the average `'body_mass_g'` for every combination of `'species'` and `'sex'`.

In [28]:
penguins.pivot_table(
    index='species',
    columns='sex',
    values='body_mass_g',
    aggfunc='mean'
)

sex,Female,Male
species,,
Adelie,3368.84,4043.49
Chinstrap,3527.21,3938.97
Gentoo,4679.74,5484.84


In [29]:
# Same information as above, but harder to read!
(
    penguins
    .groupby(['species', 'sex'])
    [['body_mass_g']]
    .mean()
)

body_mass_g
species   sex                
Adelie    Female      3368.84
          Male        4043.49
Chinstrap Female      3527.21
          Male        3938.97
Gentoo    Female      4679.74
          Male        5484.84

### Example: Finding the number of penguins per `'island'` and `'species'`

In [30]:
penguins

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Dream,41.3,20.3,194.0,3550.0,Male
1,Adelie,Torgersen,38.5,17.9,190.0,3325.0,Female
2,Adelie,Dream,34.0,17.1,185.0,3400.0,Female
...,...,...,...,...,...,...,...
330,Chinstrap,Dream,46.6,17.8,193.0,3800.0,Female
331,Adelie,Dream,39.7,17.9,193.0,4250.0,Male
332,Gentoo,Biscoe,45.1,14.5,207.0,5050.0,Female


- Suppose we want to find the number of penguins in `penguins` per `'island'` and `'species'`. We can do so _without_ `pivot_table`:

In [31]:
penguins.value_counts(['island', 'species']) 

island     species  
Biscoe     Gentoo       119
Dream      Chinstrap     68
           Adelie        55
Torgersen  Adelie        47
Biscoe     Adelie        44
Name: count, dtype: int64

In [32]:
penguins.groupby(['island', 'species']).size() 

island     species  
Biscoe     Adelie        44
           Gentoo       119
Dream      Adelie        55
           Chinstrap     68
Torgersen  Adelie        47
dtype: int64

- But the data is arguably easier to interpret when we do use `pivot_table`:

In [33]:
penguins.pivot_table(
    index='species', 
    columns='island', 
    values='bill_length_mm', # Choice of column here doesn't actually matter! Why?
    aggfunc='count',
)

island,Biscoe,Dream,Torgersen
species,,,
Adelie,44.0,55.0,47.0
Chinstrap,NaN,68.0,NaN
Gentoo,119.0,NaN,NaN


- Note that there is a `NaN` at the intersection of `'Biscoe'` and `'Chinstrap'`, because there were no Chinstrap penguins on Biscoe Island.<br><small>`NaN` stands for "not a number." It is `numpy` and `pandas`' version of a null value (the regular Python null value is `None`). </small>

- We can either use the `fillna` method afterwards or the `fill_value` argument to fill in `NaN`s.

In [34]:
penguins.pivot_table(
    index='species', 
    columns='island', 
    values='bill_length_mm', 
    aggfunc='count',
    fill_value=0,
)

island,Biscoe,Dream,Torgersen
species,,,
Adelie,44,55,47
Chinstrap,0,68,0
Gentoo,119,0,0


### Granularity

- Each row of the original `penguins` DataFrame represented a single penguin, and each column represented features of the penguins.

In [35]:
penguins

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Dream,41.3,20.3,194.0,3550.0,Male
1,Adelie,Torgersen,38.5,17.9,190.0,3325.0,Female
2,Adelie,Dream,34.0,17.1,185.0,3400.0,Female
...,...,...,...,...,...,...,...
330,Chinstrap,Dream,46.6,17.8,193.0,3800.0,Female
331,Adelie,Dream,39.7,17.9,193.0,4250.0,Male
332,Gentoo,Biscoe,45.1,14.5,207.0,5050.0,Female


- What is the **granularity** of the DataFrame below?<br><small>That is, what does each row represent?</small>

In [36]:
penguins.pivot_table(
    index='species', 
    columns='island', 
    values='bill_length_mm', 
    aggfunc='count',
    fill_value=0,
)

island,Biscoe,Dream,Torgersen
species,,,
Adelie,44,55,47
Chinstrap,0,68,0
Gentoo,119,0,0


### Reshaping

- `pivot_table` reshapes DataFrames from "long" to "wide".

- `melt` unpivots a DataFrames from "wide" to "long".

- Google, the documentation, and ChatGPT are your friends!

In [37]:
penguins_wide = penguins.pivot_table(
    index='species', 
    columns='island', 
    values='bill_length_mm', 
    aggfunc='count',
    fill_value=0,
).reset_index()
penguins_wide

island,species,Biscoe,Dream,Torgersen
0,Adelie,44,55,47
1,Chinstrap,0,68,0
2,Gentoo,119,0,0


In [38]:
penguins_wide.melt(
    id_vars = 'species',    # column(s) to use as identifier variables
    value_vars = ['Biscoe', 'Dream', 'Torgersen'],   # column(s) to unpivot
)

,species,island,value
0,Adelie,Biscoe,44
1,Chinstrap,Biscoe,0
2,Gentoo,Biscoe,119
...,...,...,...
6,Adelie,Torgersen,47
7,Chinstrap,Torgersen,0
8,Gentoo,Torgersen,0


- Yes, it's fairly complicated and the naming isn't always intuitive

- No one can remember all the syntax and arguments, but you need to know that these functions exist, and roughly what they do, so you can look them up when needed.

### Merging 🚗


In [39]:
phones = pd.DataFrame().assign(
    Model=['iPhone 16', 'iPhone 16 Pro Max', 'Samsung Galaxy S24 Ultra', 'Pixel 9 Pro'],
    Price=[799, 1199, 1299, 999],
    Screen=[6.1, 6.9, 6.8, 6.3]
)
inventory = pd.DataFrame().assign(
    Handset=['iPhone 16 Pro Max', 'iPhone 16', 'Pixel 9 Pro', 'Pixel 9 Pro', 'iPhone 16', 'iPhone 15'],
    Units=[50, 40, 10, 15, 100, 5],
    Store=['Briarwood', 'Somerset', 'Arbor Hills', '12 Oaks', 'Briarwood', 'Oakland Mall']
)

### Example: Phone sales 📱

In [40]:
# The DataFrame on the left contains information about phones on the market.
# The DataFrame on the right contains information about the stock I have in my stores.
dfs_side_by_side(phones, inventory)

,Model,Price,Screen
0,iPhone 16,799,6.1
1,iPhone 16 Pro Max,1199,6.9
2,Samsung Galaxy S24 Ultra,1299,6.8
3,Pixel 9 Pro,999,6.3
,Handset,Units,Store
0,iPhone 16 Pro Max,50,Briarwood
1,iPhone 16,40,Somerset
2,Pixel 9 Pro,10,Arbor Hills
3,Pixel 9 Pro,15,12 Oaks
4,iPhone 16,100,Briarwood


- **Question**: If I sell all of the phones in my inventory, how much will I make in revenue?

- The information I need to answer the question is spread across multiple DataFrames.

- The solution is to **merge** the two DataFrames together horizontally.

- A merge is appropriate when we have two sources of information **about the same individuals** that is **linked by a common column(s)**.<br>The common column(s) are called the **join key**.<br>

### If I sell all of the phones in my inventory, how much will I make in revenue?

In [41]:
combined = phones.merge(inventory, left_on='Model', right_on='Handset') 
combined

,Model,Price,Screen,Handset,Units,Store
0,iPhone 16,799,6.1,iPhone 16,40,Somerset
1,iPhone 16,799,6.1,iPhone 16,100,Briarwood
2,iPhone 16 Pro Max,1199,6.9,iPhone 16 Pro Max,50,Briarwood
3,Pixel 9 Pro,999,6.3,Pixel 9 Pro,10,Arbor Hills
4,Pixel 9 Pro,999,6.3,Pixel 9 Pro,15,12 Oaks


In [42]:
(combined['Price'] * combined['Units']).sum() 

196785

### What just happened?

In [43]:
# Click through the presentation that appears.
show_merging_animation()

### The `merge` method

- The `merge` DataFrame method joins two DataFrames by columns or indexes.

- When using the `merge` method, the DataFrame before `merge` is the "left" DataFrame, and the DataFrame passed into `merge` is the "right" DataFrame.<br><small>In `phones.merge(inventory)`, `phones` is considered the "left" DataFrame and `inventory` is the "right" DataFrame.<br>The columns from the left DataFrame appear to the left of the columns from right DataFrame.</small>

- By default:
    - If join keys are not specified, all shared columns between the two DataFrames are used.
    - The "type" of join performed is an inner join, which is just one of many types of joins.

### Inner joins

- The default type of join that `merge` performs is an **inner join**, which keeps the **intersection** of the join keys.

<center><img src='imgs/inner-joins.png' width=300></center>

In [44]:
# The DataFrame on the far right is the merged DataFrame.
dfs_side_by_side(phones, inventory, phones.merge(inventory, left_on='Model', right_on='Handset'))

,Model,Price,Screen
0,iPhone 16,799,6.1
1,iPhone 16 Pro Max,1199,6.9
2,Samsung Galaxy S24 Ultra,1299,6.8
3,Pixel 9 Pro,999,6.3
,Handset,Units,Store
0,iPhone 16 Pro Max,50,Briarwood
1,iPhone 16,40,Somerset
2,Pixel 9 Pro,10,Arbor Hills
3,Pixel 9 Pro,15,12 Oaks
4,iPhone 16,100,Briarwood


- Note that `'Samsung Galaxy S24 Ultra'` and `'iPhone 15'` do not appear in the merged DataFrame.

- That's because there is no `'Samsung Galaxy S24 Ultra'` in the right DataFrame (`inventory`), and no `'iPhone 15'` in the left DataFrame (`phones`).

### Other join types

- We can change the type of join performed by changing the `how` argument in `merge`.

In [45]:
phones.merge(inventory, left_on='Model', right_on='Handset', how='left')

,Model,Price,Screen,Handset,Units,Store
0,iPhone 16,799,6.1,iPhone 16,40.0,Somerset
1,iPhone 16,799,6.1,iPhone 16,100.0,Briarwood
2,iPhone 16 Pro Max,1199,6.9,iPhone 16 Pro Max,50.0,Briarwood
3,Samsung Galaxy S24 Ultra,1299,6.8,NaN,NaN,NaN
4,Pixel 9 Pro,999,6.3,Pixel 9 Pro,10.0,Arbor Hills
5,Pixel 9 Pro,999,6.3,Pixel 9 Pro,15.0,12 Oaks


In [46]:
phones.merge(inventory, left_on='Model', right_on='Handset', how='right')

,Model,Price,Screen,Handset,Units,Store
0,iPhone 16 Pro Max,1199.0,6.9,iPhone 16 Pro Max,50,Briarwood
1,iPhone 16,799.0,6.1,iPhone 16,40,Somerset
2,Pixel 9 Pro,999.0,6.3,Pixel 9 Pro,10,Arbor Hills
3,Pixel 9 Pro,999.0,6.3,Pixel 9 Pro,15,12 Oaks
4,iPhone 16,799.0,6.1,iPhone 16,100,Briarwood
5,NaN,NaN,NaN,iPhone 15,5,Oakland Mall


In [47]:
phones.merge(inventory, left_on='Model', right_on='Handset', how='outer')

,Model,Price,Screen,Handset,Units,Store
0,Pixel 9 Pro,999.0,6.3,Pixel 9 Pro,10.0,Arbor Hills
1,Pixel 9 Pro,999.0,6.3,Pixel 9 Pro,15.0,12 Oaks
2,Samsung Galaxy S24 Ultra,1299.0,6.8,NaN,NaN,NaN
3,NaN,NaN,NaN,iPhone 15,5.0,Oakland Mall
4,iPhone 16,799.0,6.1,iPhone 16,40.0,Somerset
5,iPhone 16,799.0,6.1,iPhone 16,100.0,Briarwood
6,iPhone 16 Pro Max,1199.0,6.9,iPhone 16 Pro Max,50.0,Briarwood


- The website [pandastutor.com](https://pandastutor.com) can help visualize DataFrame operations like these.<br>[**Here's a direct link**](https://pandastutor.com/vis.html#code=%23%20The%20code%20below%20just%20initializes%20the%0A%23%20phones%20and%20inventory%20DataFrames%20as%20we've%20seen%20them%0A%23%20in%20lecture.%0A%23%20---%0Aimport%20pandas%20as%20pd%0Aimport%20io%0A%0Aphones%20%3D%20pd.read_csv%28io.StringIO%28'''%0AModel,Price,Screen%0AiPhone%2016,799,6.1%0AiPhone%2016%20Pro%20Max,1199,6.9%0ASamsung%20Galaxy%20S24%20Ultra,1299,6.8%0APixel%209%20Pro,999,6.3%0A'''%29%29%0A%0Ainventory%20%3D%20pd.read_csv%28io.StringIO%28'''%0AHandset,Units,Store%0AiPhone%2016%20Pro%20Max,50,Briarwood%0AiPhone%2016,40,Somerset%0APixel%209%20Pro,10,Arbor%20Hills%0APixel%209%20Pro,15,12%20Oaks%0AiPhone%2016,100,Briarwood%0AiPhone%2015,5,Oakland%20Mall%0A'''%29%29%0A%0A%23%20---%0A%0A%23%20Below,%20uncomment%20the%20line%20that%20you'd%20like%20to%20visualize!%0A%23%20phones.merge%28inventory,%20left_on%3D'Model',%20right_on%3D'Handset',%20how%3D'inner'%29%0A%0Aphones.merge%28inventory,%20left_on%3D'Model',%20right_on%3D'Handset',%20how%3D'left'%29%0A%0A%23%20phones.merge%28inventory,%20left_on%3D'Model',%20right_on%3D'Handset',%20how%3D'right'%29%0A%0A%23%20phones.merge%28inventory,%20left_on%3D'Model',%20right_on%3D'Handset',%20how%3D'outer'%29&d=2024-09-11&lang=py&v=v1) to this specific example.

<center><img src="imgs/pt-screenshot.png" width=600></center>

### Different join types handle mismatches differently

<center><img src='imgs/all-joins.png' width=400></center>

- **Inner join**: Keep **only** the matching keys (intersection).

- **Outer join**: Keep **all** keys in both DataFrames (union).

- **Left join**: Keep all keys in the left DataFrame, whether or not they are in the right DataFrame.

- **Right join**: Keep all keys in the right DataFrame, whether or not they are in the left DataFrame.<br><small> Note that `a.merge(b, how='left')` contains the same information as `b.merge(a, how='right')`, just in a different order.</small>

### Tip: Set differences

- A set in Python is a data structure containing **unique**, **unordered** elements.

In [48]:
phones['Model']

0                   iPhone 16
1           iPhone 16 Pro Max
2    Samsung Galaxy S24 Ultra
3                 Pixel 9 Pro
Name: Model, dtype: object

In [49]:
left = set(phones['Model'])
left

{'Pixel 9 Pro', 'Samsung Galaxy S24 Ultra', 'iPhone 16', 'iPhone 16 Pro Max'}

In [50]:
inventory['Handset']

0    iPhone 16 Pro Max
1            iPhone 16
2          Pixel 9 Pro
3          Pixel 9 Pro
4            iPhone 16
5            iPhone 15
Name: Handset, dtype: object

In [51]:
right = set(inventory['Handset'])
right

{'Pixel 9 Pro', 'iPhone 15', 'iPhone 16', 'iPhone 16 Pro Max'}

- To quickly check _which_ join key values are in the left DataFrame but not the right, or vice versa, create sets out of the join keys and use the `difference` method.

In [52]:
left.difference(right) 

{'Samsung Galaxy S24 Ultra'}

In [53]:
right.difference(left) 

{'iPhone 15'}


### Notes on the `merge` method

- `merge` is flexible – you can merge using a combination of columns, or the index of the DataFrame.

-  If the two DataFrames have the same column names, `pandas` will add `_x` and `_y` to the duplicated column names to avoid having columns with the same name (change these the `suffixes` argument).

- Ensure key uniqueness on the side that should be unique

- **Best practice**: Keep the larger / more fine-grained data on the left, and always do left join
    - Preserves all rows from your primary (“left”) dataset
    - Makes data loss explicit (unmatched rows become NaN), which you can audit
    - If you just need an inner join, drop NaN afterwards
    - Easier to debug and safer for reproducibility